<a href="https://colab.research.google.com/github/shijuc/CGAN_Demo/blob/master/CGAN_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from __future__ import division, print_function
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
import itertools
import time
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import cv2
from PIL import Image
from sklearn.metrics import confusion_matrix
from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D, BatchNormalization, Dropout, Conv2D, Conv2DTranspose, AveragePooling2D, MaxPooling2D, UpSampling2D, Input, Reshape, Lambda, Concatenate, Subtract, Reshape, multiply
from keras.layers import Embedding, ZeroPadding2D
from keras.regularizers import l2
from keras.layers import LeakyReLU
from keras import backend as K
from keras.optimizers import Nadam, Adam, SGD
from keras.metrics import categorical_accuracy, binary_accuracy
from keras.callbacks import Callback, History
import tensorflow as tf
import pandas as pd
import glob
import PIL
from sys import getsizeof
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from matplotlib.ticker import MaxNLocator
from sklearn.utils import class_weight
from keras.datasets import mnist

In [12]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# Image shape information
img_rows = X_train.shape[1]
img_cols = X_train.shape[2]

if len(X_train.shape) == 4:
    channels = X_train.shape[3]
else:
    channels = 1
img_shape = (img_rows, img_cols, channels)
num_classes = 10
latent_dim = 100
optimizer = Adam(0.0002, 0.5)

In [3]:
def discriminator():
    model = Sequential()
    model.add(Dense(512, input_dim=np.prod(img_shape)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    label = Input(shape=(1,), dtype='int32')

    label_embedding = Flatten()(Embedding(num_classes, np.prod(img_shape))(label))
    flat_img = Flatten()(img)

    model_input = multiply([flat_img, label_embedding])

    validity = model(model_input)

    return Model([img, label], validity)

In [4]:
def generator():
    model = Sequential()
    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))
    model.summary()

    noise = Input(shape=(latent_dim,))
    label = Input(shape=(1,), dtype='int32')
    label_embedding = Flatten()(Embedding(num_classes, latent_dim)(label))

    model_input = multiply([noise, label_embedding])
    img = model(model_input)

    return Model([noise, label], img)

In [ ]:
discriminator = discriminator()

discriminator.compile(loss=['binary_crossentropy'], optimizer=optimizer, metrics=['accuracy'])

# Build the generator
generator = generator()

# The generator takes noise and the target label as input
# and generates the corresponding digit of that label

noise = Input(shape=(latent_dim,))
label = Input(shape=(1,))
img = generator([noise, label])

# For the combined model we will only train the generator
discriminator.trainable = False

# The discriminator takes generated image as input and determines validity
# and the label of that image
valid = discriminator([img, label])

# The combined model  (stacked generator and discriminator)
# Trains generator to fool discriminator

combined = Model([noise, label], valid)
combined.compile(loss=['binary_crossentropy'], optimizer=optimizer)

In [6]:
def sample_images(epoch):
    r, c = 2, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    sampled_labels = np.arange(0, 10).reshape(-1, 1)
    for i in range(0, len(sampled_labels)):
        print(sampled_labels[i])
    gen_imgs = generator.predict([noise, sampled_labels])

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    cnt = 0
    for i in range(2 * 5):
      plt.axis('off')
      plt.imshow(gen_imgs[cnt,:,:,0], cmap='gray')
      cnt += 1
      filename = "/content/%d" % sampled_labels[cnt-1]

      if not os.path.exists(filename):
          os.makedirs(filename)
      plt.savefig("/content/%d/%d.png" % ( sampled_labels[cnt-1], epoch))

    plt.close()


In [7]:
epochs=5000
batch_size=32
sample_interval=500

In [8]:
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

y_train = y_train.reshape(-1, 1)

# Adversarial ground truths
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

# Declaring empty lists to save the losses for plotting
d_loss_plot = []
g_loss_plot = []
acc_plot = []

In [ ]:
for epoch in range(epochs):
    #Training the Discriminator
    # Select a random half batch of images
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs, labels = X_train[idx], y_train[idx]

    # Sample noise as generator input
    noise = np.random.normal(0, 1, (batch_size, 100))

    # Generate a half batch of new images
    gen_imgs = generator.predict([noise, labels])

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch([imgs, labels], valid)
    d_loss_fake = discriminator.train_on_batch([gen_imgs, labels], fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    #Training the Generator
    # Condition on labels
    sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)

    # Train the generator
    g_loss = combined.train_on_batch([noise, sampled_labels], valid)

    # Saving the Discriminator and Generator losses and accuracy for plotting
    d_loss_plot.append(d_loss[0])
    g_loss_plot.append(g_loss)
    acc_plot.append(d_loss[1])

    # Plot the progress every 500 epochs
    if epoch % 500 == 0:
        print ("%d/%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, epochs, d_loss[0], 100*d_loss[1], g_loss))
        samples = 10
        z = np.random.normal(loc=0, scale=1, size=(samples, latent_dim))
        labels = np.arange(0, 10).reshape(-1, 1)
        x_fake = generator.predict([z, labels])
        cnt=0
        for k in range(samples):
            plt.subplot(2, 5, k+1)
            plt.imshow(x_fake[k].reshape(28, 28), cmap='gray')
            #plt.axes.axes.set_title("Digit: %d" % sampled_labels[cnt])
            plt.xticks([])
            plt.yticks([])
            cnt+=1
        plt.tight_layout()
        plt.show()

    # Saving generated image samples at every sample interval
    if epoch % sample_interval == 0:
        sample_images(epoch)

In [ ]:
plt.plot(acc_plot)

plt.title('Discriminator accuracy')

plt.ylabel('accuracy')

plt.xlabel('epoch')

plt.show()



# Loss plots

plt.plot(d_loss_plot)

plt.plot(g_loss_plot)

plt.title('Losses')

plt.ylabel('loss')

plt.xlabel('epoch')

plt.legend(['Discriminator', 'Generator'])

plt.show()

In [ ]:
!zip -r /content/content.zip /content/*

In [13]:
from numpy import unique,argmax
from keras.datasets.mnist import load_data
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout
from keras.utils import plot_model
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np
import os

In [21]:
# Extract the contents of Zip File
import zipfile

with zipfile.ZipFile('/content/content.zip', 'r') as zip:
    zip.extractall('/content')

In [ ]:
#Loading the Mnist dataset
(x_train1,y_train1), (x_test, y_test) = load_data()


#reshaping the training and testing data
#NOTE: update the variable 'data' array value places according to the amount of mnist data needed
# Example: To have 30k mnist dataset update as data= 30000
data = 59000
x_train_mnist = x_train1[:(data)].reshape((data, x_train1.shape[1], x_train1.shape[2], 1))
y_train_mnist = y_train1[:(data)].reshape((data,1))


print('Train', x_train_mnist.shape, y_train_mnist.shape)


x_test = x_test.reshape(x_test.shape[0],28,28,1)
y_test = y_test.reshape(y_test.shape[0],1)
print('Test', x_test.shape, y_test.shape)

In [ ]:
from keras.preprocessing import image
#from tf.image import rgb_to_grayscale
from keras.utils import load_img
#from keras.utils import load_img
from PIL import Image

#path to file containing cgan generated images
path_to_files = "./content"
vectorized_images_x = []
vectorized_images_Y = []

for _, file_num in enumerate(os.listdir(path_to_files)):
  filename = "./content/%s" % file_num
  print(filename)
  for _, file in enumerate(os.listdir(filename)):
    path = "%s/%s" % (filename,file)
    img = load_img(path,target_size=(28,28))
    plt.imshow(img,interpolation='nearest',cmap='gray')
    #plt.show()
    img1 = tf.image.rgb_to_grayscale(
    img, name=None)
    img_array = np.expand_dims(np.array(img1), axis=2)
    img_array.shape
    img_array=img_array.reshape(28,28,1)
    img_array.shape
    vectorized_images_x.append(img_array)
    vectorized_images_Y.append(int(file_num))

np.savez("./mnistlikedataset.npz",DataX=vectorized_images_x,DataY=vectorized_images_Y)
#load and use file
import numpy as np

path = "./mnistlikedataset.npz"
with np.load(path) as data:
    X_train = data['DataX']
    Y_train = data['DataY']

#Combine Mnist data and cgan generated data
Y_train = Y_train.reshape((Y_train.shape[0], 1))
x_train = np.concatenate([x_train_mnist,X_train])
y_train = np.concatenate([y_train_mnist,Y_train])
print('Train', x_train.shape, y_train.shape)

In [24]:
#normalizing the values of pixels of images
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')


In [ ]:
fig = plt.figure(figsize=(5,3))
for i in range(15):
  ax = fig.add_subplot(2,10, i+1, xticks=[], yticks=[])
  ax.imshow(np.squeeze(x_train[i]), cmap='gray')
  ax.set_title(y_train[i])

In [ ]:
#determine the shape of the image
img_shape =x_train.shape[1:]
print (img_shape)
y_shape =y_train.shape[1:]
print (y_shape)


In [26]:
#defining the model
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=img_shape))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(48, (3,3), activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
plot_model(model, 'model.jpg' , show_shapes=True)

In [ ]:
model.compile(optimizer='adam', loss= 'sparse_categorical_crossentropy', metrics=['accuracy'])
x= model.fit(x_train, y_train, epochs=10, batch_size=128, verbose=2, validation_split=0.1)


In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
model.save("classifier_cnn.h5")
print(f'Accuracy: {accuracy*100}')

In [ ]:
image = x_test[6]
#Lets display the image which we want to predicit
plt.imshow(np.squeeze(image), cmap='gray')
plt.show


In [ ]:
image=image.reshape(1, image.shape[0], image.shape[1], image.shape[2])
p = model.predict([image])
print('predicted: {} '.format(argmax(p)))